## Parte do banco de dados

In [ ]:
# Lendo o CSV com os dados dos eventos
df = pd.read_csv('todos_os_eventos.csv')

# Conectando ao banco de dados SQLite (criará um novo arquivo se não existir)
conn = sqlite3.connect('eventos.db')
cursor = conn.cursor()

# Criando as tabelas
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Eventos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        tipo TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS DadosEventos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        evento_id INTEGER,
        data TEXT,
        localizacao TEXT,
        FOREIGN KEY(evento_id) REFERENCES Eventos(id)
    )
''')

# Inserindo os dados na tabela Eventos (sem duplicatas)
eventos_unicos = df[['Nome', 'Categoria']].drop_duplicates().rename(columns={'Categoria': 'tipo'})
eventos_unicos.to_sql('Eventos', conn, if_exists='append', index=False)

# Obtendo os IDs dos eventos inseridos
eventos_ids = pd.read_sql_query("SELECT id, nome FROM Eventos", conn)
eventos_ids = eventos_ids.set_index('nome').to_dict()['id']

# Adicionando a coluna evento_id ao DataFrame original
df['evento_id'] = df['Nome'].map(eventos_ids)

# Inserindo os dados na tabela DadosEventos
dados_eventos = df[['evento_id', 'Data', 'Local']].rename(columns={'Data': 'data', 'Local': 'localizacao'})
dados_eventos.to_sql('DadosEventos', conn, if_exists='append', index=False)

# Fechando a conexão com o banco de dados
# conn.close()

print("Dados inseridos com sucesso no banco de dados SQLite!")

Dados inseridos com sucesso no banco de dados SQLite!


Consultas:

In [ ]:
# Consulta 1: Mostrar todos os eventos com suas datas, localização e tipo de evento
print("\n--- Consulta 1: Todos os eventos com detalhes ---")
cursor.execute('''
    SELECT Eventos.nome AS Evento, DadosEventos.data AS Data, DadosEventos.localizacao AS Localização, Eventos.tipo AS Tipo
    FROM Eventos
    INNER JOIN DadosEventos ON Eventos.id = DadosEventos.evento_id;
''')
resultados = cursor.fetchall()
df_resultado1 = pd.DataFrame(resultados, columns=[coluna[0] for coluna in cursor.description])
print(df_resultado1.to_markdown(index=False, numalign='left', stralign='left'))

# Consulta 2: Mostrar os dados dos 2 eventos mais próximos de iniciar
print("\n--- Consulta 2: 2 eventos mais próximos de iniciar ---")
cursor.execute('''
    SELECT Eventos.nome AS Evento, DadosEventos.data AS Data, DadosEventos.localizacao AS Localização, Eventos.tipo AS Tipo
    FROM Eventos
    INNER JOIN DadosEventos ON Eventos.id = DadosEventos.evento_id
    ORDER BY DadosEventos.data ASC
    LIMIT 2;
''')
resultados = cursor.fetchall()
df_resultado2 = pd.DataFrame(resultados, columns=[coluna[0] for coluna in cursor.description])
print(df_resultado2.to_markdown(index=False, numalign='left', stralign='left'))

# Consulta 3: Mostrar os eventos que acontecem no Rio de Janeiro
print("\n--- Consulta 3: Eventos no Rio de Janeiro ---")
cursor.execute('''
    SELECT Eventos.nome AS Evento, DadosEventos.data AS Data, DadosEventos.localizacao AS Localização, Eventos.tipo AS Tipo
    FROM Eventos
    INNER JOIN DadosEventos ON Eventos.id = DadosEventos.evento_id
    WHERE DadosEventos.localizacao LIKE '%Rio de Janeiro%';
''')
resultados = cursor.fetchall()
df_resultado3 = pd.DataFrame(resultados, columns=[coluna[0] for coluna in cursor.description])
print(df_resultado3.to_markdown(index=False, numalign='left', stralign='left'))

# Fechando a conexão com o banco de dados
conn.close()


--- Consulta 1: Todos os eventos com detalhes ---
| Evento                                                                                              | Data                | Localização                                                         | Tipo                 |
|:----------------------------------------------------------------------------------------------------|:--------------------|:--------------------------------------------------------------------|:---------------------|
| Rock the Mountain 2024 : 2º Final de Semana                                                         | 16 Nov18 Nov        | Parque Municipal Prefeito Paulo Rattes - Petrópolis, RJ             | Festas e Shows       |
| Rock the Mountain 2024 : 1º Final de Semana                                                         | 09 Nov11 Nov        | Parque Municipal Prefeito Paulo Rattes - Petrópolis, RJ             | Festas e Shows       |
| TIM BERNARDES                                                          